Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini 1.5 Pro : PDF Reading for Grounding service.
Feedback : shins777@gmail.com.

이 Colab은 Gemini 1.5 Pro를 이용해서 PDF 파일을 곧바로 Grounding 으로 활용하는 예제입니다.
이 예제는 1M input token 이 가능한 Gemini 1.5 pro를 RAG 아키텍처에 활용하기 위한 예제입니다.

In [9]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [10]:
from IPython.display import display, Markdown


In [11]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [12]:
MODEL_NAME="gemini-1.5-pro-preview-0409"
PROJECT_ID="ai-hangsik"
REGION="us-central1"

### Gen AI 기본 설정

In [13]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


### Gemini 1.5 Pro Invoke

In [14]:
def generate(file_path:str,
             query : str) -> None:

  vertexai.init(project="ai-hangsik", location="us-central1")
  model = GenerativeModel("gemini-1.5-pro-preview-0409")

  pdf_encoded = get_obj_content(obj_loc=file_path)
  document1 = Part.from_data(data=base64.b64decode(pdf_encoded), mime_type="application/pdf")

  responses = model.generate_content(
      [document1, query],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )

  # for response in responses:
  #   print(response.text, end="")
  return responses.text


### 대용량 파일 ( 60페이지 보고서 ) 해석.

In [15]:
dir_path = "/home/admin_/rag_ai_search/contents/"
file_path = dir_path + "하나금융경영연구소_대한민국 금융소비자 보고서.pdf"
query = "설문시점이 언제이고 세대별 참여 비율을 알려주세요."
print(file_path)

outcome = generate(file_path,query )
display(Markdown(outcome))

/home/admin_/rag_ai_search/contents/하나금융경영연구소_대한민국 금융소비자 보고서.pdf


설문시점은 **2022년 6월**이며, 세대별 표본 수는 다음과 같습니다.

* **베이비부머(~’65)**: 865명 (17.3%)
* **X세대(’66~’80)**: 1,876명 (37.5%)
* **밀레니얼(’81~’95)**: 1,672명 (33.4%)
* **Z세대(’96~’02)**: 587명 (11.7%) 


### 그래프 & 이미지 복합 데이터 해석.

In [16]:
dir_path = "/home/admin_/rag_ai_search/contents/"
file_path = dir_path + "하나금융경영연구소_대한민국 금융소비자 보고서.pdf"
query = "금융 자산 포트폴리오에서 가구 평균 금융자산이 얼마이고 X 세대의 금융자산 포트폴리오의 비율을 알려주세요."
print(file_path)

outcome = generate(file_path,query )
display(Markdown(outcome))

/home/admin_/rag_ai_search/contents/하나금융경영연구소_대한민국 금융소비자 보고서.pdf


죄송하지만 제공해 주신 텍스트에 금융 자산 포트폴리오에서 X 세대의 비율에 대한 정보가 없어 답변드릴 수 없습니다. 텍스트에는 X 세대의 평균 금융자산 규모와 다른 세대에 비해 X 세대가 주로 사용하는 금융기관에 대한 정보만 포함되어 있습니다. 